In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. Indexing Phase

### (1) Load

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/construcnt_law.pdf')
documents = loader.load()

In [2]:
# 개행문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
    
documents

[Document(metadata={'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2025-03-12T17:33:04+09:00', 'moddate': '2025-03-12T17:33:04+09:00', 'source': './data/construcnt_law.pdf', 'total_pages': 69, 'page': 0, 'page_label': '1'}, page_content='법제처                                                            1                                                       국가법령정보센터 건축법 시행령   건축법 시행령 [시행 2025. 1. 21.] [대통령령 제35221호, 2025. 1. 21., 타법개정] 국토교통부 (건축정책과 - 건축제도 일반) 044-201-3762, 3763 국토교통부 (건축안전과 - 건축구조 규정 운영) 044-201-4991 국토교통부 (녹색건축과 - 건축설비ㆍ조경 규정 운영) 044-201-4753 국토교통부 (건축안전과 - 피난ㆍ마감재료 규정 운영) 044-201-4992 국토교통부 (건축정책과 - 건축감리 규정 운영) 044-201-4752 국토교통부 (건축정책과 - 위반건축물 규정 운영) 044-201-3762, 3761        제1장 총칙   제1조(목적) 이 영은 「건축법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다. [전문개정 2008. 10. 29.]   제2조(정의) 이 영에서 사용하는 용어의 뜻은 다음과 같다. <개정 2009. 7. 16., 2010. 2. 18., 2011. 12. 8., 2011. 12. 30., 2013. 3. 23., 2014. 11. 11., 2014. 11. 28., 2015. 9. 22., 2016. 1. 19., 2016. 5. 17., 2016

### (2) split

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 각 chunk의 최대 문자 수
    chunk_overlap=20 # 인접한 텍스트 조각 간 겹치는 문자 수 (기본값: 200)
                        # 텍스트 분할 구분자 우선순위 (기본값: ['\n\n', '\n', ' ', ''])
)
docs = splitter.split_documents(documents)
docs

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션',

In [6]:
len(docs)

26

### (3) Store

In [7]:
# 임베딩 모델 생성
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
# !pip install langchain-chroma

In [9]:
from langchain_chroma.vectorstores import Chroma

vector_store = Chroma.from_documents(docs, embedding_model)

- Retriever를 사용한 유사도 기반 탐색

In [ ]:
query='백설공주와 왕비 중에 누가 더 아름답나요?'

# vector store 직접 조회
retrievals = vector_store.similarity_search_with_score(query)
retrievals


[(Document(id='fcb3d25a-5e5b-4963-84b0-ee056e814838', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  0.8401051759719849),
 (Document(id='4ed969fb-90aa-4434-a25b-74dff36a7036', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 2, 'page_label': '3', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  0.8604382276535034),
 

In [12]:
# Retriever를 사용한 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':3}
)

retriever_result = retriever.batch([query])
retriever_result

[[Document(id='fcb3d25a-5e5b-4963-84b0-ee056e814838', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  Document(id='4ed969fb-90aa-4434-a25b-74dff36a7036', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 2, 'page_label': '3', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  Document(id='98afc885-9f2e-4e4d-83a7-fe16e9f4

# 2. Retrieval and Generation Phase

### (1) 프롬프트 생성
- 사용자 질의 + 검색된 문서

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사입니다. 질문하는 아이에게 최대한 호응하면서 contetxt 기반으로만 답변해 주세요.'),
    ('user',"""
     어린이의 질문에 context만을 이용해 답변하세요.
     질문: {query}
     context: {context}
     """)    
])


In [14]:
prompt.invoke({'query':query, 'context': retrievals})

ChatPromptValue(messages=[SystemMessage(content='당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사입니다. 질문하는 아이에게 최대한 호응하면서 contetxt 기반으로만 답변해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n     어린이의 질문에 context만을 이용해 답변하세요.\n     질문: 백설공주와 왕비 중에 누가 더 아름답나요?\n     context: [(Document(id='fcb3d25a-5e5b-4963-84b0-ee056e814838', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'), 0.8401051759719849), (Document(id='4ed969fb-90aa-4434-a25b-74dff36a7036', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 2, 'pa

### (2) 모델 생성

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

### (3) Chain 생성

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# context 생성
query = '백설공주가 마지막에 만난 사람은 누구인가요?'
retrievals = retriever.batch([query])
context_text = '\n'.join([doc.page_content for doc in retrievals[0]])
#  {'query': RunnablePassthrough(), 'context': context_text} |
chain =  prompt | model | StrOutputParser()

### 사용

In [18]:
chain.invoke({'query':query, 'context': context_text})

'백설공주가 마지막에 만난 사람은 그녀를 사랑하는 왕자님이에요! 왕자님이 백설공주를 보고 너무 기뻐했답니다. 그들의 이야기는 정말 아름답고 행복해요!'